In [7]:
# import numpy and pandas
import numpy as np
import pandas as pd

In [8]:
x = np.array([10, 20, 30, 40])

In [9]:
print(x)

[10 20 30 40]


In [10]:
pd.Series(x)
x = pd.Series([5, 10, 15, 20, 25])
print(x)

0     5
1    10
2    15
3    20
4    25
dtype: int64


In [1]:
import pandas as pd

data = {
    'Department': ['HR', 'HR', 'Tech', 'Tech', 'Sales', 'Sales'],
    'Employee': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank'],
    'Salary': [50000, 52000, 70000, 73000, 45000, 47000],
    'Experience': [2, 3, 5, 6, 1, 2]
}

df = pd.DataFrame(data)
print(df)


  Department Employee  Salary  Experience
0         HR    Alice   50000           2
1         HR      Bob   52000           3
2       Tech  Charlie   70000           5
3       Tech    David   73000           6
4      Sales      Eva   45000           1
5      Sales    Frank   47000           2


In [3]:
# Using groupby() with Custom Aggregation
# 🔹 1. Apply a Custom Function
# Let’s calculate range of salaries (max - min) in each department:

def salary_range(series):
    return series.max() - series.min()

df.groupby('Department')['Salary'].agg(salary_range)


Department
HR       2000
Sales    2000
Tech     3000
Name: Salary, dtype: int64

In [4]:
df.groupby('Department')['Salary'].agg(lambda x: x.max() - x.min())


Department
HR       2000
Sales    2000
Tech     3000
Name: Salary, dtype: int64

In [5]:
df.groupby('Department')['Salary'].agg([
    'mean',
    'max',
    'min',
    lambda x: x.max() - x.min()
])

,mean,max,min,<lambda_0>
Department,,,,
HR,51000.0,52000,50000,2000
Sales,46000.0,47000,45000,2000
Tech,71500.0,73000,70000,3000


In [7]:
# Rename the custom function:

df.groupby('Department')['Salary'].agg([
    ('Average', 'mean'),
    ('Range', lambda x: x.max() - x.min())
])


,Average,Range
Department,,
HR,51000.0,2000
Sales,46000.0,2000
Tech,71500.0,3000


In [10]:
# Custom Aggregation for Multiple Columns


def weighted_salary(x):
    return (x['Salary'] * x['Experience']).sum() / x['Experience'].sum()

df.groupby('Department').agg({
    'Salary': 'mean',
    'Experience': 'mean',
    ('WeightedSalary', weighted_salary)
})


SyntaxError: ':' expected after dictionary key (359284067.py, line 10)

In [11]:
def custom_row_agg(group):
    total_salary = group['Salary'].sum()
    avg_exp = group['Experience'].mean()
    return pd.Series({
        'Total_Salary': total_salary,
        'Avg_Experience': avg_exp
    })

df.groupby('Department').apply(custom_row_agg)


/tmp/ipykernel_18282/3487719840.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Department').apply(custom_row_agg)


,Total_Salary,Avg_Experience
Department,,
HR,102000.0,2.5
Sales,92000.0,1.5
Tech,143000.0,5.5


<!-- Recap: When to Use What?
Use Case	Use This
One column, one custom logic	groupby()[col].agg(func)
One group, multiple columns involved	groupby().apply(custom_func)
Pivot view with custom agg	pivot_table(aggfunc=func)
Combine built-in and custom functions	agg([func1, func2, custom]) -->


In [12]:
# High-Level Overview
# Function	Used With	Returns	Row-wise or Group-wise?	When to Use
# agg()	Series, DataFrame, groupby()	Scalar values	Group-wise or column-wise	When you want summary statistics (mean, max, custom)
# apply()	Series, DataFrame, groupby()	Series, scalar, or DataFrame	Both	When applying a custom function across rows or columns
# transform()	Series, DataFrame, groupby()	Series (same shape)	Group-wise	When you need a group-wise computation and want to keep original shape
# map()	Series only	Series	Element-wise	Apply a function or dict to each value in a Series
# applymap()	DataFrame only	DataFrame	Element-wise	Apply a function element-wise to every cell in the DataFrame


#  1. agg() – Aggregation
# Summarizes values (reduces rows to one per group).


df.groupby('Department').agg({'Salary': 'mean', 'Experience': 'sum'})


,Salary,Experience
Department,,
HR,51000.0,5
Sales,46000.0,3
Tech,71500.0,11


In [15]:
# 2. apply() – Custom logic, flexible return

df.groupby('Department').apply(lambda group: group['Salary'].max() - group['Salary'].min())


/tmp/ipykernel_18282/2757610981.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('Department').apply(lambda group: group['Salary'].max() - group['Salary'].min())


Department
HR       2000
Sales    2000
Tech     3000
dtype: int64

In [16]:
# 3. transform() – Broadcast results

df.groupby('Department')['Salary'].transform('mean')


0    51000.0
1    51000.0
2    71500.0
3    71500.0
4    46000.0
5    46000.0
Name: Salary, dtype: float64

In [18]:
df['Gender'] = df['Name'].map({'Alice': 'F', 'Bob': 'M'})


KeyError: 'Name'

In [26]:
# 5. applymap() – Element-wise on DataFrame

# Want to apply function to every value in a DataFrame? → applymap()


# Function	Operates On	Output Shape	Example Use
# .agg()	column/group	1 row per group	Group summary (mean, sum, etc.)
# .apply()	row/group/col	Flexible	Custom calculations using multiple columns
# .transform()	column/group	Same shape	Add group stats back to each row
# .map()	Series	Same shape	Clean or map Series values
# .applymap()	DataFrame	Same shape	Modify every cell


df.applymap(lambda x: len(str(x)))


/tmp/ipykernel_18282/1786890692.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(lambda x: len(str(x)))


,Department,Employee,Salary,Experience
0,2,5,5,1
1,2,3,5,1
2,4,7,5,1
3,4,5,5,1
4,5,3,5,1
5,5,5,5,1


In [ ]:
# Function	Operates On	Output Shape	Example Use
# .agg()	column/group	1 row per group	Group summary (mean, sum, etc.)
# .apply()	row/group/col	Flexible	Custom calculations using multiple columns
# .transform()	column/group	Same shape	Add group stats back to each row
# .map()	Series	Same shape	Clean or map Series values
# .applymap()	DataFrame	Same shape	Modify every cell

In [ ]:


# Most Common Window Functions in Pandas:
# Function	Description
# rolling()	Moving window calculations (e.g., moving average)
# expanding()	Expanding window from the start to current row
# shift()	Shifts data forward or backward by n rows
# cumsum()	Cumulative sum
# rank()	Assigns a rank to values (with ties and order)



# Window function in pandas 

In [27]:
import pandas as pd

data = {
    'Day': pd.date_range(start='2024-06-01', periods=6, freq='D'),
    'Sales': [100, 120, 130, 90, 160, 150]
}
df = pd.DataFrame(data)
print(df)


         Day  Sales
0 2024-06-01    100
1 2024-06-02    120
2 2024-06-03    130
3 2024-06-04     90
4 2024-06-05    160
5 2024-06-06    150


In [29]:
# rolling() – Moving Window Calculation
# Think: sliding window of size N


df['Rolling_Mean'] = df['Sales'].rolling(window=3).mean()
df

,Day,Sales,Rolling_Mean
0,2024-06-01,100,NaN
1,2024-06-02,120,NaN
2,2024-06-03,130,116.666667
3,2024-06-04,90,113.333333
4,2024-06-05,160,126.666667
5,2024-06-06,150,133.333333


In [34]:
# df['Rolling_Mean'] = df['Sales'].rolling(3).sum()
# df['Rolling_Mean'] = df['Sales'].rolling(3).max()
df['Rolling_Mean'] = df['Sales'].rolling(3).apply(lambda x: x.std())

df


,Day,Sales,Rolling_Mean
0,2024-06-01,100,NaN
1,2024-06-02,120,NaN
2,2024-06-03,130,15.275252
3,2024-06-04,90,20.816660
4,2024-06-05,160,35.118846
5,2024-06-06,150,37.859389


In [36]:
# expanding() – Cumulative Window from Start


df['Expanding_Mean'] = df['Sales'].expanding().mean()
# What this does:

# Starts at the first row and includes all previous rows in calculation.

# Useful for tracking running averages

df

,Day,Sales,Rolling_Mean,Expanding_Mean
0,2024-06-01,100,NaN,100.000000
1,2024-06-02,120,NaN,110.000000
2,2024-06-03,130,15.275252,116.666667
3,2024-06-04,90,20.816660,110.000000
4,2024-06-05,160,35.118846,120.000000
5,2024-06-06,150,37.859389,125.000000


In [38]:
# 3. shift() – Shift Rows

df['Yesterday_Sales'] = df['Sales'].shift(1)

df

,Day,Sales,Rolling_Mean,Expanding_Mean,Yesterday_Sales
0,2024-06-01,100,NaN,100.000000,NaN
1,2024-06-02,120,NaN,110.000000,100.0
2,2024-06-03,130,15.275252,116.666667,120.0
3,2024-06-04,90,20.816660,110.000000,130.0
4,2024-06-05,160,35.118846,120.000000,90.0
5,2024-06-06,150,37.859389,125.000000,160.0


In [ ]:
# Shifts all values down by 1 row:

# Today’s row will have yesterday’s sales.

# Useful for time difference, calculating growth or deltas.

In [43]:
# 4. cumsum() – Cumulative Sum

df['Cumulative_Sales'] = df['Sales'].cumsum()
df

#  Adds up all previous values including the current row.

# First value = same as first row

# Second = first + second, and so on

# 🧠 Similarly:

# .cumprod() → cumulative product

# .cummax() → running max

# .cummin() → running min

,Day,Sales,Rolling_Mean,Expanding_Mean,Yesterday_Sales,Cumulative_Sales
0,2024-06-01,100,NaN,100.000000,NaN,100
1,2024-06-02,120,NaN,110.000000,100.0,220
2,2024-06-03,130,15.275252,116.666667,120.0,350
3,2024-06-04,90,20.816660,110.000000,130.0,440
4,2024-06-05,160,35.118846,120.000000,90.0,600
5,2024-06-06,150,37.859389,125.000000,160.0,750


In [45]:
#  5. rank() – Rank Rows

df['Rank'] = df['Sales'].rank()

df


,Day,Sales,Rolling_Mean,Expanding_Mean,Yesterday_Sales,Cumulative_Sales,Rank
0,2024-06-01,100,NaN,100.000000,NaN,100,2.0
1,2024-06-02,120,NaN,110.000000,100.0,220,3.0
2,2024-06-03,130,15.275252,116.666667,120.0,350,4.0
3,2024-06-04,90,20.816660,110.000000,130.0,440,1.0
4,2024-06-05,160,35.118846,120.000000,90.0,600,6.0
5,2024-06-06,150,37.859389,125.000000,160.0,750,5.0


In [ ]:
# Assigns ranking based on column values.

# Lowest number gets rank 1.

# Ties will get the average rank.

# .rank(ascending=False, method='dense')
# .rank(method='first')

In [47]:
df['Rank'] = df['Sales'].rank(ascending=False, method='dense')
df

,Day,Sales,Rolling_Mean,Expanding_Mean,Yesterday_Sales,Cumulative_Sales,Rank
0,2024-06-01,100,NaN,100.000000,NaN,100,5.0
1,2024-06-02,120,NaN,110.000000,100.0,220,4.0
2,2024-06-03,130,15.275252,116.666667,120.0,350,3.0
3,2024-06-04,90,20.816660,110.000000,130.0,440,6.0
4,2024-06-05,160,35.118846,120.000000,90.0,600,1.0
5,2024-06-06,150,37.859389,125.000000,160.0,750,2.0


In [48]:
df['Rank'] = df['Sales'].rank(method='first')
df

,Day,Sales,Rolling_Mean,Expanding_Mean,Yesterday_Sales,Cumulative_Sales,Rank
0,2024-06-01,100,NaN,100.000000,NaN,100,2.0
1,2024-06-02,120,NaN,110.000000,100.0,220,3.0
2,2024-06-03,130,15.275252,116.666667,120.0,350,4.0
3,2024-06-04,90,20.816660,110.000000,130.0,440,1.0
4,2024-06-05,160,35.118846,120.000000,90.0,600,6.0
5,2024-06-06,150,37.859389,125.000000,160.0,750,5.0


In [ ]:
# When to Use What?
# Use Case	Use This
# Moving averages, moving std	rolling()
# Running average from beginning	expanding()
# Lag/lead values (time series)	shift()
# Total running sales	cumsum()
# Rank employees/sales/orders/etc.	rank()

In [50]:
# Reading in Chunks using read_csv(..., chunksize=...)

# Why?
# If the dataset is too large to load into memory at once, reading it in chunks helps to process it incrementally.

import pandas as pd

chunk_iter = pd.read_csv('/home/prashantksi200/Desktop/pythonLearning/winemag-data-130k-v2.csv', chunksize=100000)  # reads 100K rows at a time

for chunk in chunk_iter:
    # Process each chunk independently
    print(chunk.head())  # or process and save the result


   Unnamed: 0   country                                        description  \
0           0     Italy  Aromas include tropical fruit, broom, brimston...   
1           1  Portugal  This is ripe and fruity, a wine that is smooth...   
2           2        US  Tart and snappy, the flavors of lime flesh and...   
3           3        US  Pineapple rind, lemon pith and orange blossom ...   
4           4        US  Much like the regular bottling from 2012, this...   

                          designation  points  price           province  \
0                        Vulkà Bianco      87    NaN  Sicily & Sardinia   
1                            Avidagos      87   15.0              Douro   
2                                 NaN      87   14.0             Oregon   
3                Reserve Late Harvest      87   13.0           Michigan   
4  Vintner's Reserve Wild Child Block      87   65.0             Oregon   

              region_1           region_2         taster_name  \
0              

In [ ]:
# Other method= Options:
# Method	Description
# 'average'	Default. Ties get the average of ranks.
# 'min'	Ties get the minimum rank.
# 'max'	Ties get the maximum rank.
# 'first'	Ties get ranks based on first appearance in the data.
# 'dense'	Like min, but no gaps in the ranking sequence.

In [51]:
# Part 2: Reducing Memory Usage with astype()
# Let’s create a DataFrame with suboptimal memory usage:


import pandas as pd
import numpy as np

data = {
    'UserID': np.arange(1, 100001),                      # int64
    'Age': np.random.randint(18, 90, 100000),            # int64 (but we don’t need full 64-bit)
    'Gender': np.random.choice(['Male', 'Female'], 100000),  # object
    'Income': np.random.rand(100000) * 100000            # float64
}

df = pd.DataFrame(data)

print(df.info(memory_usage='deep'))  # see the actual memory usage



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   UserID  100000 non-null  int64  
 1   Age     100000 non-null  int64  
 2   Gender  100000 non-null  object 
 3   Income  100000 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 8.2 MB
None


In [52]:
# Optimize it step by step

# Downcast Age to smallest possible int type
df['Age'] = df['Age'].astype('int8')  # good for 0–127 or -128 to 127

# Downcast Income to float32
df['Income'] = df['Income'].astype('float32')

# Convert Gender to category
df['Gender'] = df['Gender'].astype('category')

# Downcast UserID if it fits
df['UserID'] = pd.to_numeric(df['UserID'], downcast='unsigned')

# Check memory savings
print(df.info(memory_usage='deep'))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   UserID  100000 non-null  uint32  
 1   Age     100000 non-null  int8    
 2   Gender  100000 non-null  category
 3   Income  100000 non-null  float32 
dtypes: category(1), float32(1), int8(1), uint32(1)
memory usage: 976.9 KB
None


In [53]:
df

,UserID,Age,Gender,Income
0,1,83,Female,48836.003906
1,2,73,Female,24676.851562
2,3,33,Female,26652.703125
3,4,88,Male,93786.164062
4,5,52,Female,85588.093750
...,...,...,...,...
99995,99996,54,Male,10402.997070
99996,99997,23,Male,77387.617188
99997,99998,56,Female,10006.387695
99998,99999,81,Female,90309.976562


In [1]:
import pandas as pd
from ydata_profiling import ProfileReport

# Load a sample dataset
df = pd.read_csv("https://raw.githubusercontent.com/mwaskom/seaborn-data/master/tips.csv")

# Generate the profile report
profile = ProfileReport(df, title="Tips Data Profiling Report", explorative=True)

# Save to HTML
profile.to_file("tips_profile_report.html")


/home/prashantksi200/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/prashantksi200/.local/lib/python3.10/site-packages/ydata_profiling/profile_report.py:365: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 302.84it/s]
